# COURSERA CAPSTONE - WEEK 3 - Exercise 2 - Latitude and Longitude

In the next cell we have create Postal Code Dataframe (from Exercise 1)

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']
# instantiate the dataframe
postalCodes = pd.DataFrame(columns=column_names)

soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

trs = soup.findAll('tr')

for tr in trs:
    tds = tr.findAll('td')
    if len(tds) == 3:
        col1= tds[0].contents[0]
        col2 = tds[1].findAll("a", attrs={'title':True})
        if len(col2) > 0:
            col2=col2[0].text
            col3 = tds[2].findAll("a", attrs={'title': True})
            if len(col3) == 0:
                col3 = col2
            else:
                col3=col3[0].text
            postalCodes = postalCodes.append({'PostalCode': col1, 'Borough': col2, 'Neighborhood':col3}, ignore_index= True)

# print Postal Code Unique count to compare to final dataframe shape
print('Postal Code Unique count ', len(postalCodes['PostalCode'].unique()))
            
df = postalCodes.groupby(['PostalCode', 'Borough']).Neighborhood
postalCodes = df.agg(dict(Neighborhood=lambda x: list(x), number='count'))
postalCodes.reset_index(inplace=True)
postalCodes.drop('number', 1, inplace=True)
postalCodes.head()

Postal Code Unique count  100


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Scarborough, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


I've tried to use geocoder, but only received None as response

In [3]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-2.5           |   py36h1ba5d50_0         643 KB
    conda-4.6.3                |           py36_0         872 KB  conda-forge
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    openssl-1.1.1a             |    h14c3975_1000         4.0 MB  conda-forge
    libarchive-3.3.3           |       h5d8350f_5         1.5 MB
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    libssh2-1.8.0              |                1         239 KB  conda-forge
    python-3.6.8               |       h0371630_0      

In [2]:
import geocoder # import geocoder

def tryGeocoder(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print('coordenadas ', lat_lng_coords)
    return (lat_lng_coords)

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'

It was not possible to get latitude and longitude from geocoder, so I get them from csv

In [3]:
geoDf = pd.read_csv('https://cocl.us/Geospatial_data')
geoDf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In the next cell, merge latitude and longitude to postal code dataframe and show 30 first rows

In [4]:
geoDf.rename(columns={'Postal Code': 'PostalCode'}, inplace= True)
postalCodes = pd.merge(postalCodes, geoDf, on='PostalCode')
postalCodes.head(30)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Scarborough, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[Scarborough, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Scarborough]",43.692657,-79.264848
